In [ ]:
%pip install selenium
import selenium
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By

%pip install pandas
import pandas as pd

%pip install requests
import requests

from bs4 import BeautifulSoup
from itertools import repeat
from pprint import pprint
import time

#맥 m1에서 사용하기 위해
%pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())


driver = webdriver.Chrome()
#화면이 작을 경우 잘리는 데이터가 있을 수 있으므로 화면 크기 Max
driver.maximize_window()

driver.get('https://www.melon.com/index.htm')
time.sleep(2)
#게임에 불러올 가수 데이터
singerList = ['블랙핑크', '르세라핌', '뉴진스', '아이브', '데이식스']
#가수별로 받아올 노래 데이터
singer_song_list = []

for i in singerList:
    #검색창에 순서대로 가수 입력
    elemSearch = driver.find_element('id','top_search')
    elemSearch.send_keys(i)
    #검색버튼 누르기
    elem = driver.find_element(By.CLASS_NAME, 'search_m')
    elem.click()
    time.sleep(1)

    #태그 중 곡 리스트 카테고리 들어가기
    elem = driver.find_element(By.LINK_TEXT, '곡')
    elem.click()
    time.sleep(1)

    #가수 제목 리스트에 미리 추가
    singer_song_list.append(i)
    #첫 번째 페이지의 노래 제목들 리스트
    elem = driver.find_elements(By.CSS_SELECTOR, '#frm_defaultList > div > table > tbody > tr > td:nth-child(3) > div > div > a.fc_gray')
    #리스트 안에서 노래 하나씩 뽑아서 전체 데이터에 추가
    for e in elem:
        linkText = e.get_property('textContent')
        #회색처리 된 부분 제외
        if linkText != None:
            #괄호 안의 정보 제거
            if '(' in linkText:
                linkText = linkText[:linkText.index('(')]
                singer_song_list.append({i : linkText,})
            else:
                singer_song_list.append({i : linkText,})

    #두 번째 페이지부터는 for문으로 돌리기 
    pages = driver.find_elements(By.CSS_SELECTOR, '#pageObjNavgation > div > span > *')
    for j in range(2, len(pages)+1):
        elem = driver.find_element(By.CSS_SELECTOR, '#pageObjNavgation > div > span > a:nth-child({})'.format(j))
        elem.click()
        time.sleep(2)
        elem = driver.find_elements(By.CSS_SELECTOR, '#frm_defaultList > div > table > tbody > tr > td:nth-child(3) > div > div > a.fc_gray')
        for e in elem:
            linkText = e.get_property('textContent')
            if linkText != None:
                if '(' in linkText:
                    linkText = linkText[:linkText.index('(')]
                    singer_song_list.append({i : linkText,})
                else:
                    singer_song_list.append({i : linkText,})

    #검색어 초기화
    searchBox = driver.find_element(By.XPATH, '//*[@id="top_search"]')
    searchBox.clear()

    #pandas를 통해서 데이터 저장
    df = pd.DataFrame(singer_song_list)
    df.to_csv("singer_song_list.csv")
